In [ ]:
import os
import json
import math
import glob
import itertools
from datetime import datetime
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Optional deps
try:
    import statsmodels.api as sm
    import statsmodels.formula.api as smf
except Exception:
    sm = None
    smf = None

try:
    from scipy import stats
except Exception:
    stats = None

YEARS = list(range(2018, 2026))            
ARTICLES_DIR_TEMPLATE = "articles_{year}_new"
ARTICLES_FILE = "all_articles_enhanced.jsonl"

TOP_N_PAIRS = 50          
BRIDGE_QUANTILE = 0.90   
MIN_YEARS_PER_PAIR = 3   
N_PERM = 3000             

def find_latest_dir(prefix: str):
    cands = [d for d in os.listdir(".") if d.startswith(prefix) and os.path.isdir(d)]
    if not cands: return None
    cands.sort(reverse=True)
    return cands[0]

def ensure_outdirs():
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    root = f"paper_ready_{ts}"
    figdir = os.path.join(root, "figures")
    tbldir = os.path.join(root, "tables")
    os.makedirs(figdir, exist_ok=True)
    os.makedirs(tbldir, exist_ok=True)
    return root, figdir, tbldir

def load_partA():
    a_dir = find_latest_dir("field_convergence_A_")
    if not a_dir: raise FileNotFoundError("Missing field_convergence_A_* folder.")
    fca_path = os.path.join(a_dir, "fca_summary.csv")
    w_year_path = os.path.join(a_dir, "per_year_field_weights.csv")
    if not os.path.exists(fca_path): raise FileNotFoundError(f"Missing {fca_path}")
    if not os.path.exists(w_year_path): raise FileNotFoundError(f"Missing {w_year_path}")
    df_fca = pd.read_csv(fca_path)
    df_wy = pd.read_csv(w_year_path)
    df_fca["pair"] = list(map(lambda xy: tuple(sorted((xy[0], xy[1]))),
                              zip(df_fca["field_i"], df_fca["field_j"])))
    df_wy["pair"] = list(map(lambda xy: tuple(sorted((xy[0], xy[1]))),
                             zip(df_wy["field_i"], df_wy["field_j"])))
    return a_dir, df_fca, df_wy

def load_partB():
    b_dir = find_latest_dir("bridge_emergence_B_")
    if not b_dir: raise FileNotFoundError("Missing bridge_emergence_B_* folder.")
    bes_path = os.path.join(b_dir, "bes_summary.csv")
    per_year_part_path = os.path.join(b_dir, "per_year_author_participation.csv")
    if not os.path.exists(bes_path): raise FileNotFoundError(f"Missing {bes_path}")
    if not os.path.exists(per_year_part_path): raise FileNotFoundError(f"Missing {per_year_part_path}")
    df_bes = pd.read_csv(bes_path)
    df_bes["author_id"] = df_bes["author_id"].astype(str)
    return b_dir, df_bes

def load_partC():
    c_dir = find_latest_dir("novelty_AB_")
    # Part C is helpful for CPR aggregations and scatter/box figures (optional)
    # If not present, we will recompute what we need
    return c_dir

def copy_if_exists(src, dst):
    try:
        if src and os.path.exists(src):
            import shutil
            shutil.copy2(src, dst)
            return True
    except Exception:
        pass
    return False

def top_bottom_pairs(df_fca, n=TOP_N_PAIRS):
    df_fca = df_fca[df_fca["years_covered"] >= MIN_YEARS_PER_PAIR].copy()
    df_sorted = df_fca.sort_values("FCA_slope", ascending=False)
    top = df_sorted.head(n).copy()
    bottom = df_sorted.tail(n).copy()
    return top, bottom

def permutation_test_spearman(x, y, n_perm=3000, seed=7):
    """
    Two-sided permutation test for Spearman correlation.
    Returns corr_obs, p_perm
    """
    x = np.asarray(x, float)
    y = np.asarray(y, float)
    mask = np.isfinite(x) & np.isfinite(y)
    x = x[mask]; y = y[mask]
    if len(x) < 5:
        return np.nan, np.nan
    if stats is None:
        # Fallback: no permutation
        corr = pd.Series(x).corr(pd.Series(y), method="spearman")
        return float(corr), np.nan
    corr_obs, _ = stats.spearmanr(x, y)
    rng = np.random.default_rng(seed)
    more_extreme = 0
    for _ in range(n_perm):
        y_perm = rng.permutation(y)
        c, _ = stats.spearmanr(x, y_perm)
        if abs(c) >= abs(corr_obs):
            more_extreme += 1
    p = (more_extreme + 1) / (n_perm + 1)
    return float(corr_obs), float(p)

def mannwhitney_and_cliffs_delta(a, b):
    """
    Returns MW U p-value and Cliff's delta.
    """
    a = np.asarray([v for v in a if np.isfinite(v)], float)
    b = np.asarray([v for v in b if np.isfinite(v)], float)
    if len(a) == 0 or len(b) == 0:
        return np.nan, np.nan
    if stats is None:
        return np.nan, np.nan
    u_stat, p = stats.mannwhitneyu(a, b, alternative="two-sided")
    # Cliff's delta
    # Efficient approximation using ranks:
    # delta = (2*U/(n_a*n_b)) - 1
    delta = (2 * u_stat / (len(a)*len(b))) - 1
    return float(p), float(delta)

def plot_pair_trends(df_wy, pair, figdir):
    """
    Draws a line chart of weight_cosine over years for a given field pair.
    Saves to figures/.
    """
    sub = df_wy[(df_wy["pair"] == pair) & (df_wy["year"].isin(YEARS))].copy()
    if sub.empty:
        return None
    sub = sub.groupby("year", as_index=False)["weight_cosine"].mean().sort_values("year")
    plt.figure(figsize=(6.2,4.2))
    plt.plot(sub["year"], sub["weight_cosine"], marker="o")
    plt.xlabel("Year")
    plt.ylabel("Cosine weight (co-occurrence)")
    plt.title(f"Trend: {pair[0]} × {pair[1]}")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    fname = f"pair_trends_field_weights_{pair[0].replace('/','-')}__{pair[1].replace('/','-')}.png"
    path = os.path.join(figdir, fname)
    plt.savefig(path, dpi=140)
    plt.close()
    return path

def main():
    root, figdir, tbldir = ensure_outdirs()

    # Load A/B/C
    a_dir, df_fca, df_wy = load_partA()
    b_dir, df_bes = load_partB()
    c_dir = load_partC()

    # Load link file from C if exists (contains bridge shares & CPR per pair)
    df_link = None
    if c_dir:
        link_path = os.path.join(c_dir, "link_fca_bes_per_pair.csv")
        if os.path.exists(link_path):
            df_link = pd.read_csv(link_path)
            # ensure numeric
            for col in ["FCA_slope", "share_bridge_authors", "cpr_mean", "cpr_median"]:
                if col in df_link.columns:
                    df_link[col] = pd.to_numeric(df_link[col], errors="coerce")

    # 1) Correlation (FCA vs Bridge share) + permutation p-value
    corr_rows = []
    if df_link is not None and "share_bridge_authors" in df_link.columns:
        corr_obs, p_perm = permutation_test_spearman(df_link["FCA_slope"], df_link["share_bridge_authors"], n_perm=N_PERM)
        corr_rows.append({"metric_x":"FCA_slope","metric_y":"share_bridge_authors","spearman":corr_obs,"p_perm":p_perm})
    else:
        # fallback: compute from scratch minimally (share not trivial → skip)
        corr_rows.append({"metric_x":"FCA_slope","metric_y":"share_bridge_authors","spearman":np.nan,"p_perm":np.nan})
    pd.DataFrame(corr_rows).to_csv(os.path.join(tbldir,"correlation_summary.csv"), index=False, encoding="utf-8")

    # 2) GLM: share_bridge_authors ~ FCA_slope + weight_mean + years_covered (with weights = authors_total)
    if sm is not None and smf is not None and df_link is not None and {"authors_total","authors_bridge","FCA_slope","weight_mean","year_first","year_last"}.issubset(df_link.columns):
        # Construct response as proportion with weights
        df_glm = df_link.copy()
        df_glm["share"] = df_glm["authors_bridge"] / df_glm["authors_total"].replace(0, np.nan)
        df_glm["years_range"] = df_glm["year_last"] - df_glm["year_first"] + 1
        df_glm = df_glm.replace([np.inf,-np.inf], np.nan).dropna(subset=["share","FCA_slope","weight_mean","years_range","authors_total"])
        if len(df_glm) >= 10:
            # Binomial GLM with weights (number of trials)
            # Use quasi-binomial approximation via GLM Binomial + robust SE
            endog = pd.DataFrame({"success": df_glm["authors_bridge"], "failure": df_glm["authors_total"] - df_glm["authors_bridge"]})
            df_glm["success"] = df_glm["authors_bridge"]
            df_glm["total"]   = df_glm["authors_total"]
            formula = "success ~ FCA_slope + weight_mean + years_range"
            model = smf.glm(formula=formula, data=df_glm, family=sm.families.Binomial(), freq_weights=df_glm["total"])
            res = model.fit(cov_type="HC3")
            summ = res.summary2().tables[1].reset_index().rename(columns={"index":"term"})
            summ.to_csv(os.path.join(tbldir,"glm_bridge_share_logit.csv"), index=False, encoding="utf-8")
            try:
                with open(os.path.join(tbldir,"glm_bridge_share_logit.tex"), "w", encoding="utf-8") as f:
                    f.write(summ.to_latex(index=False, float_format="%.4f"))
            except Exception:
                pass
        else:
            pd.DataFrame({"note":["Not enough rows for GLM"]}).to_csv(os.path.join(tbldir,"glm_bridge_share_logit.csv"), index=False, encoding="utf-8")
    else:
        pd.DataFrame({"note":["statsmodels not available or link file missing"]}).to_csv(os.path.join(tbldir,"glm_bridge_share_logit.csv"), index=False, encoding="utf-8")

    # 3) Impact tests (Top vs Bottom FCA) using CPR
    impact_rows = []
    if df_link is not None and "cpr_mean" in df_link.columns:
        df_sorted = df_link.sort_values("FCA_slope", ascending=False)
        df_top = df_sorted.head(TOP_N_PAIRS)
        df_bot = df_sorted.tail(TOP_N_PAIRS)
        # Use per-paper CPR aggregated in Part C (pair_paper_cpr collapsed). Here we use pair-level means as proxy.
        a = df_top["cpr_mean"].values
        b = df_bot["cpr_mean"].values
        # Mann-Whitney + Cliff's delta
        p_mw, cliffs = mannwhitney_and_cliffs_delta(a, b)
        impact_rows.append({"n_top":len(a), "n_bottom":len(b),
                            "mean_top":np.nanmean(a), "mean_bottom":np.nanmean(b),
                            "delta_mean":np.nanmean(a)-np.nanmean(b),
                            "mannwhitney_p":p_mw, "cliffs_delta":cliffs})
    else:
        impact_rows.append({"note":"No CPR in link file; run Part C first."})
    pd.DataFrame(impact_rows).to_csv(os.path.join(tbldir,"impact_tests.csv"), index=False, encoding="utf-8")

    # 4) Figures: copy from C if exists, else redraw scatter/box quickly from df_link
    # scatter
    scatter_src = os.path.join(c_dir, "plots","scatter_fca_vs_bridge_share.png") if c_dir else None
    if not copy_if_exists(scatter_src, os.path.join(figdir,"scatter_fca_vs_bridge_share.png")) and df_link is not None:
        plt.figure(figsize=(7,5))
        plt.scatter(df_link["FCA_slope"], df_link["share_bridge_authors"])
        plt.xlabel("FCA_slope (field convergence acceleration)")
        plt.ylabel("Share of bridge authors")
        plt.title("FCA vs Bridge share")
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(os.path.join(figdir,"scatter_fca_vs_bridge_share.png"), dpi=140)
        plt.close()

    # box
    box_src = os.path.join(c_dir, "plots","box_impact_top_vs_bottom.png") if c_dir else None
    copy_if_exists(box_src, os.path.join(figdir,"box_impact_top_vs_bottom.png"))
    vio_src = os.path.join(c_dir, "plots","violin_impact_top_vs_bottom.png") if c_dir else None
    copy_if_exists(vio_src, os.path.join(figdir,"violin_impact_top_vs_bottom.png"))

    # 5) Field-pair trends: w_ij(t) lines for Top 10 FCA
    top10, _bottom = top_bottom_pairs(df_fca, n=10)
    # Table of top pairs (LaTeX)
    try:
        top_tbl = top10[["field_i","field_j","FCA_slope","weight_first","weight_last","years_covered"]].copy()
        top_tbl.columns = ["Field i","Field j","FCA slope","Weight (first)","Weight (last)","Years"]
        with open(os.path.join(tbldir,"top_fca_pairs.tex"), "w", encoding="utf-8") as f:
            f.write(top_tbl.to_latex(index=False, float_format="%.4f"))
    except Exception:
        pass
    # Plots
    for _, r in top10.iterrows():
        pair = tuple(sorted((r["field_i"], r["field_j"])))
        plot_pair_trends(df_wy, pair, figdir)

    # 6) Findings (short narrative)
    findings = []
    # correlation
    try:
        corr_df = pd.read_csv(os.path.join(tbldir,"correlation_summary.csv"))
        spearman = corr_df.loc[0,"spearman"]
        p_perm   = corr_df.loc[0,"p_perm"]
        findings.append(f"[Correlation] Spearman(FCA, Bridge Share) = {spearman:.3f} (perm p={p_perm:.4f})")
    except Exception:
        findings.append("[Correlation] Not available.")
    # glm
    if os.path.exists(os.path.join(tbldir,"glm_bridge_share_logit.csv")):
        glm_df = pd.read_csv(os.path.join(tbldir,"glm_bridge_share_logit.csv"))
        if "term" in glm_df.columns:
            row = glm_df[glm_df["term"]=="FCA_slope"]
            if not row.empty and "Coef." in glm_df.columns:
                coef = float(row["Coef."].values[0])
                findings.append(f"[GLM] FCA_slope coefficient (logit) ≈ {coef:.3f} (robust SE in table).")
    # impact
    try:
        imp_df = pd.read_csv(os.path.join(tbldir,"impact_tests.csv"))
        if "delta_mean" in imp_df.columns:
            dm = float(imp_df["delta_mean"].values[0])
            p  = float(imp_df["mannwhitney_p"].values[0]) if "mannwhitney_p" in imp_df.columns else np.nan
            cd = float(imp_df["cliffs_delta"].values[0]) if "cliffs_delta" in imp_df.columns else np.nan
            findings.append(f"[Impact] ΔCPR(Top-Bottom) = {dm:.4f}, MW p={p:.4f}, Cliff's δ={cd:.3f}")
    except Exception:
        pass

    with open(os.path.join(root,"findings.txt"), "w", encoding="utf-8") as f:
        f.write("\n".join(findings))

if __name__ == "__main__":
    main()
